# Init

In [1]:
INIT = False
GET_GIT_LIB = False

if INIT:
    !pip install dolphins_recognition_challenge

if GET_GIT_LIB:
    !git clone https://github.com/luciantin-ml/delfini
    !mv ./delfini/lib ./
    #  !rm -rf ./delfini

# Libs

In [1]:
import optuna
from lib.dataLoader import *
from lib.models import *
from lib.train import train_model
from dolphins_recognition_challenge.datasets import display_batches

# Data loaders

In [2]:
data_loader, data_loader_test = get_data_loader(8)

# display_batches(data_loader, n_batches=8)
# display_batches(data_loader_test, n_batches=8)

# Setup

In [7]:
from lib import models
from lib import train

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model = getattr(models, "model_fasterrcnn_resnet50")(128, 0.5)
paramss = [p for p in model.parameters() if p.requires_grad]
model.to(device)

optimizer = torch.optim.SGD(paramss, lr=paramss["lr"], momentum=paramss["momentum"], weight_decay=paramss["weight_decay"])

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.001)

train.train_model_without_optim(1, model, optimizer, scheduler, device, data_loader, data_loader_test)


AttributeError: 'NoneType' object has no attribute 'conv5_mask'

# Train

In [4]:
import optuna
import numpy as np
from lib.engine import Engine
# TODO image loader batch
def objective(trial):
    params = {"num_epochs": trial.suggest_int("num_epochs", 10, 50, 2), "model": trial.suggest_categorical("model", ["model_maskrcnn_resnet50"]), "hidden_layer_size": trial.suggest_categorical("hidden_layer_size", [32, 64, 128, 256, 512]), "box_score_thresh": trial.suggest_float("box_score_thresh", 0.4, 0.6, step=0.1, log=False), "optimizer": trial.suggest_categorical("optimizer", ["SGD", "Adam"]), "lr": trial.suggest_float("lr", 1e-6, 1e-3), "weight_decay": trial.suggest_float("weight_decay", 0.00001, 0.0004, step=0.00001, log=False)}

    if params["optimizer"] == "SGD":
        params["momentum"] = trial.suggest_float("momentum", 0.06, 0.16, step=0.02, log=False)

    params["scheduler"] = trial.suggest_categorical("scheduler", ["StepLR", "MultiStepLR", "ReduceLROnPlateau"])

    if params["scheduler"] == "StepLR" :
        params["step_size"] = trial.suggest_int("step_size", 2, 20, 1)
        params["gamma"] = trial.suggest_float("gamma", 0.02, 0.16, step=0.02, log=False)

    if params["scheduler"] == "MultiStepLR" :
        params["gamma"] = trial.suggest_float("gamma", 0.02, 0.16, step=0.02, log=False)

        min_milestone = trial.suggest_int("min_milestone", 1, 10, 2)
        max_milestone = trial.suggest_int("max_milestone", min_milestone, params["num_epochs"], 2)
        step_milestone = trial.suggest_int("step_milestone", 1, 10, 2)
        params["milestones"] = np.arange(min_milestone, max_milestone, step_milestone)

    if params["scheduler"] == "ReduceLROnPlateau" :
        params["factor"] = trial.suggest_float("factor", 0.02, 0.16, step=0.02, log=False)
        params["patience"] = trial.suggest_int("patience", 2, 10, 1)
        params["cooldown"] = trial.suggest_int("cooldown", 2, 10, 1)
    eng = Engine(trial, params, data_loader, data_loader_test)
    return eng.train()

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}

[I 2021-05-01 17:52:16,336] A new study created in memory with name: no-name-744031ad-9a28-4d0f-87bc-e26191d81275


<class 'dict'>
['__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']
True
<class 'dict'>
Epoch #0


KeyboardInterrupt: 

In [ ]:
# from dolphins_recognition_challenge.instance_segmentation.model import show_prediction, iou_metric_example, \
#     train_one_epoch, show_predictions
#
# for i in range(4):
#     iou_test_image = iou_metric_example(model, data_loader_test.dataset[i], 0.5)
#     img, _ = data_loader_test.dataset[i]
#     print(f"IOU metric for the input image is: {iou_test_image}")
#     show_prediction(model, img, width=820)
#
#
# from dolphins_recognition_challenge.instance_segmentation.model import iou_metric, show_predictions_sorted_by_iou
#
# mean_iou_testset, _ = iou_metric(model, data_loader_test.dataset)
#
# print(f"Mean IOU metric for the test set is: {mean_iou_testset}")
